In [1]:
import pandas as pd
import numpy as np

In [2]:
path = 'C:/Users/ah3wj/Desktop/epi-segmentation/'
df = pd.read_csv(path + 'covid-19-data/us-counties.csv')

In [3]:
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [5]:
df.county.head()

0    Snohomish
1    Snohomish
2    Snohomish
3         Cook
4    Snohomish
Name: county, dtype: object

In [6]:
df['date'] = df['date'].astype('datetime64[ns]')

In [7]:
df.sort_values(by = ['date'], inplace = True, ascending = True)

In [8]:
all_dates = df.date.unique()
print(len(all_dates))

94


In [16]:
df_counties = df['county']  + '--' +df['state']
all_counties = df_counties.unique()
print(len(all_counties))

2836


In [13]:
def preprocess(lookup, resolution):
  with open(path + 'covid-19-data/processed-data/processed-us-counties-' + lookup + '.csv', 'w+') as lookup_file:

    with open(path + 'covid-19-data/processed-data/us-counties-order.csv', 'w+')  as order_file:

      for county in all_counties:
        print(county + '\n', end='')
        order_file.write(county + '\n')

        for date in all_dates:
          filter1 = df[resolution] == county
          filter2 =  df['date'] == date

          if((filter1 & filter2).any()):
            value = df.loc[filter1 & filter2][lookup].to_string(index=False)
          else:
            value = '0'

          if(date == all_dates[-1]):
            print(value , end='')
            lookup_file.write(value)
          else:
            print(value + ', ', end='')
            lookup_file.write(value + ', ')

        print('\n')
        lookup_file.write('\n')

In [14]:
########################### careful here 
########################################
########################################

lookup = 'cases'
resolution = 'county'

preprocess(lookup= lookup, resolution = resolution)


Snohomish
 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  3,  5,  8,  17,  18,  26,  36,  45,  61,  69,  107,  132,  154,  175,  199,  265,  309,  347,  384,  446,  480,  518,  613,  633,  778,  913,  912,  1067,  1126,  1228,  1304,  1375,  1449,  1502,  1558,  1602,  1658,  1694,  1701,  1801,  1834,  1866,  1907,  1949,  1987,  2031,  2055,  2092,  2142,  2162,  2162,  2208,  2239

Cook
0, 0, 0,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  4,  4,  4,  5,  6,  6,  7,  11,  17,  22,  27,  40,  51,  76,  76,  107,  179,  279,  412,  548,  805,  922,  1194,  1418,  1904,  2239,  2613,  3445,     1
 3727,     1
 4496,     1
 5152,     1
 5575,     1
 6473,     1
 7439,     1
 8043,     2
 8728,     4
 9509,      4
 10520,      6
 11415,      6
 12472,      6
 13417, 

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  5,  10,  12,  14,   1
 18,   1
 18,   3
 21,   3
 27,   3
 27,   3
 37,   9
 40,   9
 44,  11
 44,  14
 49,  14
 50,  14
 50,  16
 50,  18
 50,  18
 50,  20
 50,  20
 50,  20
 51,  20
 52,  22
 52,  26
 52,  26
 52,  27
 52,  27
 52,  34
 52
  1,  34
 52
  2,  34
 52
  2

Sacramento
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  1,  1,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  9,  9,  9,  16,  16,  28,  32,  32,  39,  39,  44,  52,  52,  52,  87,  87,  112,  112,  164,  164,  164,  224,  224,  314,  340,  389,  411,  441,  461,  524,  579,  612,  652,  699,  707,  739,  816,  853,  879,  896,  914,  925,  940,  954,  971,  987

Spokane
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  2,  2,  4,  5,  12,  14,  20,  37,  52,  96,  155,  269,  330,  464,  645,  1339,  2468,  4408,  6211,  9045,  12305,  14905,  20011,  23112,  25399,  30766,  33768,  38087,  43139,  47440,  51810,  57160,  63307,  67552,  68776,  76876,  81803,  87028,  92384,  98308,  103208,  106764,  110465,  118302,  123146,  127352,  131273,  134446,  136816,  139335,  142442,  145855

Unknown
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  2,  2,  2,  2,  2,  3,  3,  3,  3,  5,  5,  1
 5,   1
 20
  3,   1
  1
 20
  4,   1
  1
 20
  5
  3,   1
  2
 21
  5
  1
  1
  3,   1
  2
 23
  5
  3
  3
  1,   1
  3
  5
 33
  8
  4
  2
  3,   4
  1
  3
 44
  6
  2
 12
  3
  3
 13
  1,  11
  1
  3
  6
 14
 44
  2
  6
 14
 19
 18
  5
  3
  1,    1
  11
 151
   6
  21
  66
   8
  27
  15
  21
  26
   9
   3
   1


0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  5,  7,  7,  9,  13,  14,  15,  20,  21,  22,  23,  26,  28,  30,  35,  35,  36,  38,  38,  41,  41,  41,  41,  43,  43,  43,  43,  45,  45,  45,  44,  44,  44,  45,  45,  45,  45,  45,  46

Walla Walla
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  3,  3,  3,  4,  7,  7,  7,  8,  9,  9,  9,  12,  14,  17,  17,  17,  21,  21,  21,  22,  25,  26,  27,  28,  28,  28,  36,  36,  36,  52

Fulton
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  2,  2,  2,  2,  2,  4,  5,  5,  6,  6,  6,  6,  13,  20,  27,  33,  49,  68,   1
 95,    1
 100,    1
 111,    1
 152,    1
 191,    

   18,    26
 2998
   16
  296
    1
  177
    4
   27
    1
   18,    28
 3099
   17
  306
    1
    1
  181
    4
   27
   19,    28
 3218
   18
  313
    1
  188
    4
   27
    1
   19,    29
 3314
   18
  321
    1
  195
    4
   27
    1
    1
   19

Williamson
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  1,  2,  4,  4,  8,  9,  10,  14,  18,  21,   4
 24
  1,   8
 35
  1,   9
 35
  1,   9
 47
  1,   9
 48
  1,  14
 48
  1,  14
 64
  1,  22
 66
  1,  27
 70
  1,  34
 91
  1,  34
 95
  1,   37
 101
   1,   37
 111
   1,   42
 111
   1,   55
 148
   1,   55
 199
   1,   71
 221
   2,   71
 239
   3,   77
 258
   3,   81
 260
   3,  278
  81
   8,   87
 287
   9,  288
 103
  10,  299
 110
  10,  312
 110
  10,  312
 117
  10,  318
 119
  10,  322
 124
  10,  322
 128
  12,  324
 140
  12,  336
 146
  13,  342
 151
  13,  348
 155
  14,  156
 358
  15,  359
 158
  16,  365
 167
  17,  

...

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1

Frio
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1

Geneva
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1, 0, 0, 0,  1,  1,  1,  2,  2,  2,  4,  4,  4,  4,  5,  6

Pine
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1

Nobles
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  1,  3,  2,  2,  12,  36,  60,  76,  101,  126,  196

Trimble
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  2,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2

Hocking
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4

Kearny
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  4,  8

Alfalfa
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1

Churchill
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  1,  1,  1,  1,  1

Haskell
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  2,  2,  2
 1,  3
 1,  3
 1,  3
 6

Woods
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1,  1,  1,  3,  3,  3,  3

Red Willow
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1

Cochran
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1

Jim Hogg
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  1

Carlisle
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

**Now Time to Verify**

In [ ]:
filename = path + 'covid-19-data/processed-data/processed-us-states-' + lookup + '.csv'
verification = np.loadtxt(filename, dtype=int, delimiter=',')

print(verification)

In [ ]:
flag = False

for i in range(len(verification)):

  if(flag):
    break
    
  state = all_states[i]   
#   print(state)
  for j in range(len(verification[i])):
    if(verification[i][j] != 0):
      date = all_dates[j]

#       print(date)
      
      df_of_state = df[df[resolution]== state]
      df_of_time = df_of_state[df_of_state['date'] == date]
      value = int(df_of_time[lookup].to_string(index=False))
      
      assert value == verification[i][j]
      if( value != verification[i][j]):
        print("Doesn't match", state, date)
        print('verif index: ', j, 'value: ', verification[i][j])
        print('actual: ', value)
        flag = True
        break
        
